# Chi Square Test


https://passel2.unl.edu/view/lesson/9beaa382bf7e/8#:~:text=If%20your%20chi%2Dsquare%20calculated,to%20reject%22%20your%20null%20hypothesis.

If your chi-square calculated value is greater than the chi-square critical value, then you reject your null hypothesis.
Rejecting hypothesis means we assume something other than chance was at play.

In [1]:
chi_square_value=6.35 # value for 0.01 configence (should only occur in 0.01 of all cases other than by chance)

In [2]:
import pandas as pd
#df = pd.read_parquet('sources/metadata.parquet', engine='pyarrow')
#df_large = pd.read_parquet('sources/metadata-large.parquet', engine='pyarrow')

exact_matches = pd.read_parquet("../results/artists_exact_match_large.parquet", engine='pyarrow')
assert "artists" in exact_matches.columns, f'artists is not in {exact_matches.columns}'
assert "num_artists" in exact_matches.columns, f'num_artists is not in {exact_matches.columns}'

print(f'exact match shapes {exact_matches.shape}')

# read excel_artist_names
import my_utils

excel_artist_names = my_utils.read_lines_as_list("../sources/excel_artists_copy_paste_name.txt")
hundred_artist_names = excel_artist_names[0:100]
ten_artist_names = excel_artist_names[0:10]


artist_mentions = pd.read_parquet('../results/artist_mentions.parquet', engine='pyarrow')

exact match shapes (1819808, 5)


In [8]:
import wordcloud
from collections import defaultdict
import re

def read_stopwords():
    # from https://gist.github.com/sebleier/554280
    return my_utils.read_lines_as_list("stopwords.txt")

def compute_freq(df, col, sw):
    # print(sw)
    freq = defaultdict(lambda: 0)
    for i, row in df.iterrows():
        string = row[col]
        for s in re.split("[, \-!?:]+", string):
            s=s.strip().lower()
            if s not in sw:
                if len(s)>0:
                    if s == "cannonsiin":
                        print(f'before {s} {freq[s]}')
                    
                    freq[s]=freq[s]+1
                    if s == "cannonsiin":
                        print(f'after {s} {freq[s]}')
    return freq



In [4]:
stopwords=read_stopwords()
stopwords.append("greg")
stopwords.append("rutkowski")

freqs = compute_freq(exact_matches, "prompt", stopwords)
print(f'finished')

greg_df = my_utils.exact_match_dataframe(exact_matches, "greg rutkowski")
freqs_greg = compute_freq(greg_df, "prompt", stopwords)
print(f'finished')

finished
finished


In [5]:

import chi_values as cv

print(f'{len(freqs.keys())} items in plain, {len(freqs_greg.keys())} items in greg')

chi_values = cv.chi_value_dataframe(freqs, freqs_greg)

print(f'amount of queries for greg {greg_df.shape[0]}')
print(greg_df.head().prompt.values)

print(f'amount of words with higher than threshold chi value: {chi_values[chi_values["chi_values"]>=chi_square_value].shape[0]}')

print(chi_values.head(40))


221758 items in plain, 46543 items in greg
amount of queries for greg 180080
['rimworld concept art isometric perspective, epic, by victo ngai, kilian eng vibrant colours, dynamic lighting, digital art, winning award masterpiece, fantastically beautiful, illustration, inspired by beksinski and dan mumford, trending on artstation, by greg rutkowski, 8 k '
 'realistic streetview, retro futuristic suburban scene, brutal architecture, straight edged buildings, street signs, detailed shop facades, neon signs, video screens, traffic lights, street lights, pedestrians, jodhpurs greg manchess painting by sargent and leyendecker, cyberpunk 2 0 7 7 concept intricate elegant illustration forza, by greg rutkowski by greg tocchini by james gilleard '
 'street scene, brutal architecture, retro futuristic concept cars, low profile tires, detailed chrome trim, jodhpurs greg manchess painting by sargent and leyendecker, cyberpunk 2 0 7 7 concept intricate elegant illustration forza, by greg rutkowski b

# Cluster Artists by Chi Square Values

First we need to find out which words to use for the artist features.
For this we can use the words ranked by chi values for the individual artists.

In [15]:
import copy

artists = ["greg rutkowski", "artgerm", "wlop"]

stopwords=read_stopwords()
for a in artists:
    stopwords.extend(a.split(" "))
# print(stopwords)

print(f'exact_matches shape {exact_matches.shape}')

freqs_all = compute_freq(exact_matches, "prompt", stopwords)
print(f'cannonsiin counts all {freqs_all["cannonsiin"]}')
for index, artist in enumerate(artists):
    
    artist_df = my_utils.exact_match_dataframe(exact_matches, artist)
    print(artist_df.shape)
    assert artist_df.shape[0]>0
    freqs_artist = compute_freq(artist_df, "prompt", stopwords)
    
    frequencies_other = copy.deepcopy(freqs_all)
    print(f'cannonsiin counts other {freqs_all["cannonsiin"]}')
    for key, value in frequencies_other.items():
        #if value == 0:
            #print(f'{key} {value}')
        assert value > 0, f'value must be bigger than 0 since we go over all prompts, word {key}'
        assert frequencies_other[key] >= freqs_artist[key], f'frequency of {key} in entire should be higher or equal to specific, {frequencies_other[key]} {freqs_artist[key]}'
        frequencies_other[key] = max(frequencies_other[key] - freqs_artist[key], 0)
    
    artist_chi_df = cv.chi_value_dataframe(frequencies_other, freqs_artist)
    #chi_df.drop(["word_frequency"], axis=1, inplace=True)
    artist_chi_df.rename({"word_frequency": f'frequency_{artist}',"chi_values": f'chi_values_{artist}', "word": f'word_{artist}'}, axis='columns', inplace=True)
    
    if index == 0:
        chi_df = artist_chi_df.copy()
    else:
        chi_df = pd.concat([chi_df, artist_chi_df], axis=1)

print(chi_df.head())



exact_matches shape (1819808, 5)
before cannonsiin 0
after cannonsiin 1
cannonsiin counts all 1
(180080, 6)
before cannonsiin 0
after cannonsiin 1
cannonsiin counts other 1
(136220, 6)
cannonsiin counts other 1
(63075, 6)
cannonsiin counts other 1
       word     chi_values  word_frequency      word     chi_values  \
0  alphonse  162005.420188           52558  alphonse  167279.675874   
1     mucha  142250.798336           53139     mucha  161666.017410   
2       art   82915.079017          174496     focus   93792.212552   
3     focus   61307.562108           63441   stanley   81499.852025   
4     sharp   59990.473292           66179     sharp   76748.249468   

   word_frequency          word    chi_values  word_frequency  
0           47178  riabovitchev  66067.235944            5251  
1           49381           lau  55952.187030            6401  
2           62927        andrei  55129.329196            5297  
3           13805         tooth  53856.656015            4803  
4    